In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

%cd ../..
!pwd

import os
import glob
import argparse
import pandas as pd
import numpy as np
import warnings
from datetime import datetime, timedelta
import gspread
from google.oauth2.service_account import Credentials

from modules.download import *
from modules.indicators import *
from modules.pipeline import *
from modules.tools import *
from modules.simulation import *
from modules.google_sheet_tools import *

warnings.simplefilter(action="ignore", category=FutureWarning)

/Users/ivanosipchyk/dev/investing/ema-crossover
/Users/ivanosipchyk/dev/investing/ema-crossover


In [53]:
sheet_url = "https://docs.google.com/spreadsheets/d/16HZfFIu37ZG7kRgLDI9SqS5xPhb3pn3iY2ubOymVseU/edit?gid=489311250#gid=489311250"

DEFAULT_FILTERS = [
    "Above_EMA_34",
    "Above_EMA_50",
    "Above_EMA_200",
    "MACD_Positive",
    # "MACD_Signal_Negative",
    "Volume",
    "Price",
    "ATR",
]

In [18]:
stock_data = download_data(['COR', 'FERG', 'HIG', 'TXNM', 'NJR', 'AROC'], period='200d', interval='1d')
stock_data_labeled = apply_to_dict(stock_data, process_symbol_df)

Processing symbols:   0%|          | 0/6 [00:00<?, ?it/s]

In [19]:
trades_df = generate_trades(stock_data_labeled)

In [20]:
trades_df

,Date,Symbol,Entry Price,Shares,Position Size,Stop Loss,Risk,Risk per Share,BE Price,BE Level,Position Left,Realized,Unrealized,Stop Rule,Offset,Max TP,Skip TP
0,2025-09-24,COR,302.93,0.33,100.0,290.00,4.27,12.93,302.93,315.86,1.0,None,None,crossover,0.0,5,1
1,2025-09-24,HIG,132.46,0.75,100.0,130.00,1.86,2.46,132.46,134.92,1.0,None,None,crossover,0.0,5,1
2,2025-09-24,NJR,47.34,2.11,100.0,46.32,2.15,1.02,47.34,48.36,1.0,None,None,crossover,0.0,5,1
3,2025-09-24,AROC,26.00,3.85,100.0,24.56,5.54,1.44,26.00,27.44,1.0,None,None,crossover,0.0,5,1


In [ ]:
# write_trades_to_sheet(trades_df, sheet_url)

In [22]:
def read_trades_from_sheet(sheet_url: str, tab_name: str = "Trades") -> pd.DataFrame:
    """
    Read trades from a Google Sheets tab into a pandas DataFrame.

    Args:
        sheet_url (str): Google Sheets URL.
        tab_name (str): Name of the worksheet tab to read. Defaults to "Trades".

    Returns:
        pd.DataFrame: DataFrame containing trades.
    """
    # --- Authenticate ---
    client = get_gs_client()

    # --- Open spreadsheet and worksheet ---
    spreadsheet = client.open_by_url(sheet_url)
    try:
        worksheet = spreadsheet.worksheet(tab_name)
    except gspread.WorksheetNotFound:
        raise RuntimeError(f"Worksheet '{tab_name}' not found in spreadsheet!")

    # --- Get all values (first row = headers) ---
    all_values = worksheet.get_all_values()
    if not all_values or len(all_values) < 2:
        print("⚠️ No trade data found.")
        return pd.DataFrame()

    headers, data = all_values[0], all_values[1:]

    # --- Build DataFrame ---
    df = pd.DataFrame(data, columns=headers)

    # --- Try convert numeric columns ---
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="ignore")  # keep strings intact

    # --- Try convert date column ---
    if "Date" in df.columns:
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

    return df

In [54]:
trades_df_read = read_trades_from_sheet(sheet_url)
trades_df_read.head()

,Date,Symbol,Entry Price,Shares,Position Size,Stop Loss,Risk,Risk per Share,BE Price,BE Level,Position Left,Realized,Unrealized,TP Reached,Stop Rule,Offset,Max TP,Skip TP
0,2025-09-24,COR,302.93,0.33,100,290.00,4.27,12.93,302.93,315.86,1,NaN,NaN,NaN,crossover,0,5,1
1,2025-09-24,HIG,132.46,0.75,100,130.00,1.86,2.46,132.46,134.92,1,NaN,NaN,NaN,crossover,0,5,1
2,2025-09-24,NJR,47.34,2.11,100,46.32,2.15,1.02,47.34,48.36,1,NaN,NaN,NaN,crossover,0,5,1
3,2025-09-24,AROC,26.00,3.85,100,24.56,5.54,1.44,26.00,27.44,1,NaN,NaN,NaN,crossover,0,5,1


In [71]:
def evaluate_trades(trades_df: pd.DataFrame, stock_data: dict) -> pd.DataFrame:
    """
    Evaluate open trades against stock data, updating realized/unrealized returns
    and TP/SL/BE/EMA exit conditions.
    """
    df = trades_df.copy()

    # Filter open trades
    open_trades = df[df["Position Left"] > 0]

    for idx, trade in open_trades.iterrows():
        symbol = trade["Symbol"]
        entry_date = pd.to_datetime(trade["Date"])
        entry_price = trade["Entry Price"]
        stop_loss = trade["Stop Loss"]
        risk = trade["Risk"]
        max_tp = int(trade["Max TP"])
        skip_tp = int(trade["Skip TP"])
        tp_coef = float(trade.get("TP Coef", 1.0))
        pos_size = trade["Shares"]

        print(f"\n🔎 Evaluating trade {idx} | {symbol} | Entry: {entry_price} | SL: {stop_loss} | Risk: {risk}")

        # --- Stock data for symbol ---
        if symbol not in stock_data:
            print(f"❌ No stock data for {symbol}, skipping...")
            continue
        df_sym = stock_data[symbol].copy()
        df_sym["Datetime"] = pd.to_datetime(df_sym["Datetime"])

        # Filter rows after entry
        df_sym = df_sym[df_sym["Datetime"] >= entry_date].reset_index(drop=True)

        # --- Build TP levels ---
        tp_levels = [entry_price + (i + 1 + skip_tp) * risk for i in range(max_tp)]
        tp_sizes = tp_distribution(max_tp, skip_tp, coef=tp_coef)

        # Track trade state
        position_left = 1.0
        realized = 0.0

        # Handle TP Reached history
        tp_reached_raw = trade.get("TP Reached", "")
        if pd.isna(tp_reached_raw) or tp_reached_raw == "":
            tp_reached = []
        elif isinstance(tp_reached_raw, (int, float)):
            tp_reached = [f"TP{int(tp_reached_raw)}"]
        else:
            tp_reached = [f"TP{int(x)}" for x in str(tp_reached_raw).split(",") if x]

        be_price = entry_price
        be_level = entry_price + risk  # first level before BE can activate

        for _, day in df_sym.iterrows():
            date, high, low, close = day["Datetime"], day["High"], day["Low"], day["Close"]

            # --- Stop Loss check ---
            if low <= stop_loss:
                realized += pos_size * position_left * stop_loss
                position_left = 0.0
                if stop_loss < entry_price:
                    tp_reached.append("SL")
                elif stop_loss > entry_price:
                    tp_reached.append("Trailing_SL")
                else:
                    tp_reached.append("BE")
                    
                print(f"🛑 {date.date()} | SL hit at {stop_loss}, closing trade.")
                break

            # --- Take Profit checks ---
            for i, tp in enumerate(tp_levels):
                tp_label = f"TP{i+1}"
                if tp_label not in tp_reached and high >= tp:
                    exit_size = tp_sizes[i]
                    realized += pos_size * exit_size * tp
                    position_left -= exit_size
                    tp_reached.append(tp_label)
                    print(f"✅ {date.date()} | {tp_label} hit at {tp}, exited {exit_size*100:.1f}% of position. Left: {position_left:.2f}")

                    # Update stop loss (trailing rule)
                    stop_loss = tp - 2 * risk if i >= 1 else be_price
                    print(f"↔️ New Stop Loss set to {stop_loss}")

                    # If last TP reached, trade is closed
                    if i + 1 == max_tp:
                        position_left = 0.0
                        print(f"🏁 Max TP{max_tp} reached, closing trade.")
                        break

            # --- EMA crossover check ---
            if day["EMA_Close_8"] < day["EMA_Close_20"]:
                realized += pos_size * position_left * close
                position_left = 0.0
                tp_reached.append("EMA_Cross")
                print(f"📉 {date.date()} | EMA crossover exit at {close}, closing trade.")
                break

        # --- Update trade row ---
        df.at[idx, "Position Left"] = position_left
        df.at[idx, "Realized"] = realized
        df.at[idx, "Unrealized"] = position_left * pos_size * df_sym.iloc[-1]["Close"]
        df.at[idx, "TP Reached"] = ",".join(tp_reached)

        print(f"📊 Final state: Realized={realized:.2f}, Unrealized={df.at[idx,'Unrealized']:.2f}, Position Left={position_left:.2f}, TP Reached={df.at[idx,'TP Reached']}")

    return df

In [72]:
trades_df_read_evaluated = evaluate_trades(trades_df_read, stock_data_labeled)
trades_df_read_evaluated.head()


🔎 Evaluating trade 0 | COR | Entry: 302.93 | SL: 290.0 | Risk: 4.27
📊 Final state: Realized=0.00, Unrealized=101.94, Position Left=1.00, TP Reached=

🔎 Evaluating trade 1 | HIG | Entry: 132.46 | SL: 130.0 | Risk: 1.86
📊 Final state: Realized=0.00, Unrealized=98.66, Position Left=1.00, TP Reached=

🔎 Evaluating trade 2 | NJR | Entry: 47.34 | SL: 46.32 | Risk: 2.15
📊 Final state: Realized=0.00, Unrealized=99.95, Position Left=1.00, TP Reached=

🔎 Evaluating trade 3 | AROC | Entry: 26.0 | SL: 24.56 | Risk: 5.54
📊 Final state: Realized=0.00, Unrealized=99.56, Position Left=1.00, TP Reached=


,Date,Symbol,Entry Price,Shares,Position Size,Stop Loss,Risk,Risk per Share,BE Price,BE Level,Position Left,Realized,Unrealized,TP Reached,Stop Rule,Offset,Max TP,Skip TP
0,2025-09-24,COR,302.93,0.33,100,290.00,4.27,12.93,302.93,315.86,1,0.0,101.9436,,crossover,0,5,1
1,2025-09-24,HIG,132.46,0.75,100,130.00,1.86,2.46,132.46,134.92,1,0.0,98.6625,,crossover,0,5,1
2,2025-09-24,NJR,47.34,2.11,100,46.32,2.15,1.02,47.34,48.36,1,0.0,99.9507,,crossover,0,5,1
3,2025-09-24,AROC,26.00,3.85,100,24.56,5.54,1.44,26.00,27.44,1,0.0,99.5610,,crossover,0,5,1


In [ ]:
def update_trades_sheet(sheet_url: str, evaluated_trades: pd.DataFrame, tab_name: str = "Trades"):
    """
    Update Google Sheet trades tab with evaluated trades.
    Only open trades (Position Left > 0) are updated; closed trades are untouched.

    Args:
        sheet_url (str): Google Sheets URL.
        evaluated_trades (pd.DataFrame): Trades after evaluation.
        tab_name (str): Worksheet tab name.
    """
    if 'Date' not in evaluated_trades.columns.to_list():
        evaluated_trades = evaluated_trades.reset_index()
        
    client = get_gs_client()
    spreadsheet = client.open_by_url(sheet_url)

    try:
        worksheet = spreadsheet.worksheet(tab_name)
    except gspread.WorksheetNotFound:
        raise RuntimeError(f"Worksheet '{tab_name}' not found!")

    # Read all existing data
    all_values = worksheet.get_all_values()
    if not all_values or len(all_values) < 2:
        print("⚠️ No data in sheet to update.")
        return

    headers, data = all_values[0], all_values[1:]
    df_sheet = pd.DataFrame(data, columns=headers)

    # Convert numeric columns in sheet
    for col in df_sheet.columns:
        df_sheet[col] = pd.to_numeric(df_sheet[col], errors="ignore")

    if "Date" in df_sheet.columns:
        df_sheet["Date"] = pd.to_datetime(df_sheet["Date"], errors="coerce")

    # --- Merge updates: only open trades ---
    # Identify trades in sheet by Symbol + Date (or another unique identifier)
    key_cols = ["Date", "Symbol"]
    df_sheet.set_index(key_cols, inplace=True)
    
    evaluated_trades.set_index(key_cols, inplace=True)

    # Update only open trades (Position Left > 0)
    for idx, row in evaluated_trades.iterrows():
        if row["Position Left"] > 0:
            if idx in df_sheet.index:
                df_sheet.loc[idx] = row  # overwrite the row
            else:
                df_sheet.loc[idx] = row  # new trade, append

    # Reset index
    df_sheet.reset_index(inplace=True)

    # --- Convert datetime columns to string ---
    for col in df_sheet.columns:
        if pd.api.types.is_datetime64_any_dtype(df_sheet[col]):
            df_sheet[col] = df_sheet[col].dt.strftime("%Y-%m-%d %H:%M:%S")

    # --- Write back to sheet ---
    all_values_to_write = [df_sheet.columns.tolist()] + df_sheet.values.tolist()
    worksheet.update(f"A1", all_values_to_write, value_input_option="USER_ENTERED")

    print(f"✅ Sheet '{tab_name}' updated with evaluated trades (open trades updated, closed trades unchanged).")

In [70]:
update_trades_sheet(sheet_url, trades_df_read_evaluated)

['index', 'Date', 'Symbol', 'Entry Price', 'Shares', 'Position Size', 'Stop Loss', 'Risk', 'Risk per Share', 'BE Price', 'BE Level', 'Position Left', 'Realized', 'Unrealized', 'TP Reached', 'Stop Rule', 'Offset', 'Max TP', 'Skip TP']


/var/folders/q7/0s4jjvb94hb_2_90j2545plw0000gn/T/ipykernel_34958/1689353169.py:61: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(f"A1", all_values_to_write, value_input_option="USER_ENTERED")


✅ Sheet 'Trades' updated with evaluated trades (open trades updated, closed trades unchanged).


In [ ]:
def main(filters, 
         symbols_list=None, 
         sheet_url="https://docs.google.com/spreadsheets/d/16HZfFIu37ZG7kRgLDI9SqS5xPhb3pn3iY2ubOymVseU/edit#gid=1171524967",
         potential_tab="Potential Setups"):
    print("⬇️ Downloading fresh data...")
    shift = 0

    # Default test symbols if none provided
    if symbols_list is None:
        symbols_list = ['XYZ', 'STT', 'POR', 'FOX', 'EVRG', 'MMYT', 'BHF', 'MGNI', 'ATRC']

    # --- Download OHLCV ---
    stock_data = download_data(
        symbols=symbols_list,
        period="500d",
        interval="1d",
        batch_size=100,
    )

    # --- Label data ---
    stock_data_labeled = apply_to_dict(stock_data, process_symbol_df)

    # --- Run potential entries ---
    potential_entries = find_potential_entries(
        stock_data_labeled,
        shift=shift,
        filters=filters,
    )

    print("\n📈 Potential entries:")
    print(potential_entries)

    # --- Save potential entries to Google Sheets ---
    if potential_entries:
        write_potential_entries(
            symbols=potential_entries,
            sheet_url=sheet_url,
            sheet_tab=potential_tab
        )

    # --- Get previous trading day's entries ---
    prev_day_entries = get_previous_trading_day_entries(
        sheet_url=sheet_url,
        sheet_tab=potential_tab
    )

    print("\n📅 Previous trading day entries:")
    print(prev_day_entries)

    # --- Filter stock data for entries ---
    stock_data_for_entries = {sym: df for sym, df in stock_data_labeled.items() if sym in prev_day_entries}

    # --- Generate trades DataFrame ---
    trades_df = generate_trades(
        stock_data_for_entries,
        stop_rule="crossover",
        sl_offset_pc=0,
        max_tp=5,
        skip_tp=0
    )

    # --- Write trades to Google Sheets ---
    if not trades_df.empty:
        write_trades_to_sheet(
            trades_df=trades_df,
            sheet_url=sheet_url,
            stop_rule="crossover",
            sl_offset_pc=0,
            max_tp=5,
            skip_tp=0,
            tp_coef=1.0,
            position_size=100
        )

    # --- Return everything ---
    return {
        "today": potential_entries,
        "previous_day": prev_day_entries,
        "trades_df": trades_df
    }

In [87]:
results = main(DEFAULT_FILTERS)

⬇️ Downloading fresh data...


Processing symbols:   0%|          | 0/9 [00:00<?, ?it/s]


📈 Potential entries:
[]

📅 Previous trading day entries:
['XYZ', 'STT', 'POR', 'FOX', 'EVRG', 'MMYT', 'BHF', 'MGNI', 'ATRC']
Created new tab: Trades_crossover_SL0_TP5_skip0_coef1.0


/var/folders/q7/0s4jjvb94hb_2_90j2545plw0000gn/T/ipykernel_29269/1146075931.py:74: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(


✅ Appended 5 trades to tab 'Trades_crossover_SL0_TP5_skip0_coef1.0' with headers


In [88]:
def read_trades_from_sheet(sheet_url, sheet_tab):
    """
    Read all trades from a Google Sheets tab and return as a DataFrame.

    Args:
        sheet_url (str): Full URL of the Google Sheet.
        sheet_tab (str): Name of the tab containing trades.

    Returns:
        pd.DataFrame: All trades from the sheet, with 'Date' parsed as datetime.
    """
    # --- Authenticate ---
    creds_path = os.environ.get("GOOGLE_SHEETS_API_CREDENTIALS")
    if creds_path is None:
        raise RuntimeError("GOOGLE_SHEETS_API_CREDENTIALS not set in environment!")

    creds = Credentials.from_service_account_file(
        creds_path,
        scopes=[
            "https://www.googleapis.com/auth/spreadsheets",
            "https://www.googleapis.com/auth/drive"
        ]
    )
    client = gspread.authorize(creds)

    # --- Open sheet & tab ---
    spreadsheet = client.open_by_url(sheet_url)
    worksheet = spreadsheet.worksheet(sheet_tab)

    # --- Get all data ---
    rows = worksheet.get_all_records()  # list of dicts
    if not rows:
        return pd.DataFrame()  # empty sheet

    df = pd.DataFrame(rows)

    # --- Parse Date column ---
    if "Date" in df.columns:
        try:
            df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")
        except Exception:
            pass  # leave as string if parsing fails

    return df

In [91]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/16HZfFIu37ZG7kRgLDI9SqS5xPhb3pn3iY2ubOymVseU/edit"
TAB_NAME = "Trades_crossover_SL0_TP5_skip0_coef1.0"

trades_df = read_trades_from_sheet(SHEET_URL, TAB_NAME)

In [92]:
trades_df

,Date,Symbol,EntryPrice,ShareN,PositionSize,StopLoss,Risk,BEPrice,Stop Rule,Offset,...,TP_1,TP_1_Size,TP_2,TP_2_Size,TP_3,TP_3_Size,TP_4,TP_4_Size,TP_5,TP_5_Size
0,2025-09-22,POR,42.92,2.33,100,42.59,33,42.92,crossover,0,...,43.25,0.03,43.58,0.06,43.91,0.13,44.24,0.26,44.57,0.52
1,2025-09-22,FOX,56.03,1.78,100,53.85,218,56.03,crossover,0,...,58.21,0.03,60.39,0.06,62.57,0.13,64.75,0.26,66.93,0.52
2,2025-09-22,MMYT,102.51,0.98,100,99.29,322,102.51,crossover,0,...,105.73,0.03,108.95,0.06,112.17,0.13,115.39,0.26,118.61,0.52
3,2025-09-22,MGNI,26.12,3.83,100,23.60,252,26.12,crossover,0,...,28.64,0.03,31.16,0.06,33.68,0.13,36.20,0.26,38.72,0.52
4,2025-09-22,ATRC,36.61,2.73,100,35.71,90,36.61,crossover,0,...,37.51,0.03,38.41,0.06,39.31,0.13,40.21,0.26,41.11,0.52


## Update Trades

In [ ]:
data = {
    "Date": ["2025-09-22"] * 5,
    "Symbol": ["POR", "FOX", "MMYT", "MGNI", "ATRC"],
    "EntryPrice": [42.92, 56.03, 102.51, 26.12, 36.61],
    "ShareN": [2.33, 1.78, 0.98, 3.83, 2.73],
    "PositionSize": [100, 100, 100, 100, 100],
    "StopLoss": [42.59, 53.85, 99.29, 23.6, 35.71],
    "Risk": [0.33, 2.18, 3.22, 2.52, 0.90],
    "BEPrice": [42.92, 56.03, 102.51, 26.12, 36.61],
    "BEActivation": [43.25, 58.21, 105.73, 28.64, 37.51],
    "Realized": ["", "", "", "", ""],
    "Unrealized": ["", "", "", "", ""],
    "Position Left": [1, 1, 1, 1, 1],
    "Stop Rule": ["crossover"] * 5,
    "Offset": [0, 0, 0, 0, 0],
    "Max TP": [5, 5, 5, 5, 5],
    "Skip TP": [0, 0, 0, 0, 0],
    "TP_1": [43.25, 58.21, 105.73, 28.64, 37.51],
    "TP_1_Size": [0.03, 0.03, 0.03, 0.03, 0.03],
    "TP_2": [43.58, 60.39, 108.95, 31.16, 38.41],
    "TP_2_Size": [0.06, 0.06, 0.06, 0.06, 0.06],
    "TP_3": [43.91, 62.57, 112.17, 33.68, 39.31],
    "TP_3_Size": [0.13, 0.13, 0.13, 0.13, 0.13],
    "TP_4": [44.24, 64.75, 115.39, 36.2, 40.21],
    "TP_4_Size": [0.26, 0.26, 0.26, 0.26, 0.26],
    "TP_5": [44.57, 66.93, 118.61, 38.72, 41.11],
    "TP_5_Size": [0.52, 0.52, 0.52, 0.52, 0.52],
    "MaxTPReached": [0, 0, 0, 0, 0]
}


In [15]:
stock_data = download_data(symbols=data['Symbol'], period='500d', interval='1d')

In [ ]:
def evaluate_trades(trades, stock_data):
    # filter open trades
    # go through each trade

        # select data for the trade and select last row
        # update be price
        # check stop loss
        # check tp levels
        # check ema crossover
        # if any new reached, update realized profit, reduce position left and calculate unrealized profit

    open_trades = trades[trades['PositionLeft'] > 0]

    for open_trade in open_trades.iterrows():
        symbol = open_trade['Symbol'].values[0]
        today_data = stock_data[symbol].iloc[-1]

        max_tp_reached = open_trade['MaxTPReached'].values[0]

        tp_cols = [col for cols in open_trade.columns if col.startswith('TP') and not col.endswith('Size')]

        


In [ ]:
def evaluate_trades(trades, stock_data):
    # filter open trades
    # go through each trade

        # select data for the trade and select last row
        # update be price
        # check stop loss
        # check tp levels
        # check ema crossover
        # if any new reached, update realized profit, reduce position left and calculate unrealized profit


In [12]:
df_trades = pd.DataFrame(data)
df_trades.head()

,Date,Symbol,EntryPrice,ShareN,PositionSize,StopLoss,Risk,BEPrice,BEActivation,Realized,...,TP_1,TP_1_Size,TP_2,TP_2_Size,TP_3,TP_3_Size,TP_4,TP_4_Size,TP_5,TP_5_Size
0,2025-09-22,POR,42.92,2.33,100,42.59,0.33,42.92,43.25,,...,43.25,0.03,43.58,0.06,43.91,0.13,44.24,0.26,44.57,0.52
1,2025-09-22,FOX,56.03,1.78,100,53.85,2.18,56.03,58.21,,...,58.21,0.03,60.39,0.06,62.57,0.13,64.75,0.26,66.93,0.52
2,2025-09-22,MMYT,102.51,0.98,100,99.29,3.22,102.51,105.73,,...,105.73,0.03,108.95,0.06,112.17,0.13,115.39,0.26,118.61,0.52
3,2025-09-22,MGNI,26.12,3.83,100,23.60,2.52,26.12,28.64,,...,28.64,0.03,31.16,0.06,33.68,0.13,36.20,0.26,38.72,0.52
4,2025-09-22,ATRC,36.61,2.73,100,35.71,0.90,36.61,37.51,,...,37.51,0.03,38.41,0.06,39.31,0.13,40.21,0.26,41.11,0.52


In [9]:
# Example today's closing prices
today_prices = {
    "POR": 43.6,
    "FOX": 61.0,
    "MMYT": 104.0,
    "MGNI": 25.0,
    "ATRC": 36.0,
}

# Example EMA cross signal (say only MGNI crossed)
ema_cross = {"POR": False, "FOX": False, "MMYT": False, "MGNI": True, "ATRC": False}

result = evaluate_trades(df_trades, today_prices, ema_cross)
print(result[["Symbol","IsOpen","Realised"]])

  Symbol  IsOpen  Realised
0    POR    True  0.115331
1    FOX    True  0.583616
2   MMYT    True  0.000000
3   MGNI   False -4.287902
4   ATRC   False  0.000000
